## 0. Import Libraries
#### _(for firebase web-app) - run only once_

In [52]:
# import firebase_admin
# from firebase_admin import credentials, db
# from datetime import datetime

# # Initialize Firebase Admin SDK
# cred = credentials.Certificate("D:\Academics\Semester VI\ES 333 - Microprocessors and Embedded Systems\PROJECT\snippet.json")
# firebase_admin.initialize_app(cred, {
#     'databaseURL': 'https://anpr-bg-default-rtdb.asia-southeast1.firebasedatabase.app/'
# })

# 1. The LLM - performs OCR
#### _(running on locally, and hosted on a server)_

In [53]:
import requests
import base64
import json
import re
from PIL import Image
import io

# Function to encode image to base64
def encode_image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image file
image_path = "license_plate_image.jpg"  # Replace with your actual image path

# Encode the image to base64
base64_image = encode_image_to_base64(image_path)

# API request payload
payload = {
    "model": "llava-phi-3-mini",  # This matches the API identifier shown in LM Studio
    "messages": [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "Extract the text on the number plate. JUST ONE TEXT STRING."},
                {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{base64_image}"}}
            ]
        }
    ],
    "temperature": 0.2,  # Lower temperature for more deterministic results
    "max_tokens": 1024,  # Adjust based on expected text length
    "stream": False
}

# Send request to LM Studio server
response = requests.post(
    "http://127.0.0.1:1234/v1/chat/completions",
    headers={"Content-Type": "application/json"},
    json=payload
)

# Process the response
if response.status_code == 200:
    result = response.json()
    full_response = result["choices"][0]["message"]["content"]
    
    # Extract text inside quotes if present
    quote_match = re.search(r'"([^"]*)"', full_response)
    if quote_match:
        extracted_text = quote_match.group(1)
    else:
        # If no quotes found, clean the text by removing explanatory phrases
        extracted_text = re.sub(r'(The license plate reads:|The license plate number is:|The text on the number plate is:|Number plate:|License plate:|The text is:|Text:)', '', full_response).strip()
    if extracted_text[2:3]=="O":
        extracted_text=extracted_text[0:2]+"0"+extracted_text[3:]
    if extracted_text[2:4]=="99":
        extracted_text=extracted_text[0:2]+"9"+extracted_text[4:]
    print(extracted_text)
else:
    print(f"Error: {response.status_code} - {response.text}")

BH01LV1947


## 2. Verifying if the String has reached WebApp

In [54]:
import firebase_admin
from firebase_admin import credentials, db
import datetime

# Path to your service account key
cred = credentials.Certificate("D:\Academics\Semester VI\ES 333 - Microprocessors and Embedded Systems\PROJECT\snippet.json")

# Initialize Firebase app
# firebase_admin.initialize_app(cred, {
#     'databaseURL': 'https://anpr-bg-default-rtdb.asia-southeast1.firebasedatabase.app/'
# # })

# Reference to 'outsiders' node
ref = db.reference('outsiders')

# Get current data and calculate next index
current_data = ref.get()

if isinstance(current_data, dict):
    next_index = str(max(map(int, current_data.keys())) + 1)
elif isinstance(current_data, list):
    next_index = str(len(current_data))
else:
    next_index = "1"

# Ask user for the new plate number
plate_number = extracted_text

# Data to upload
new_entry = {
    "entry": "12-04-2025 "+str(datetime.datetime.now().strftime("%H:%M:%S"),),
    "plate": plate_number
}

# Upload
ref.child(next_index).set(new_entry)

print(f"Entry present at {next_index}: {new_entry}")

<>:6: SyntaxWarning: invalid escape sequence '\A'
<>:6: SyntaxWarning: invalid escape sequence '\A'
C:\Users\Akshat\AppData\Local\Temp\ipykernel_19564\2066299066.py:6: SyntaxWarning: invalid escape sequence '\A'
  cred = credentials.Certificate("D:\Academics\Semester VI\ES 333 - Microprocessors and Embedded Systems\PROJECT\snippet.json")


Entry present at 26: {'entry': '12-04-2025 11:03:46', 'plate': 'BH01LV1947'}


## 3. Sending Detected String to STM

In [55]:
import serial
import time

# Make sure vehicle_number is defined
# You should use the license plate detected from your OCR code
vehicle_number = extracted_text  # This should be defined in a previous cell

# Open COM8 with 115200 baud
ser = serial.Serial('COM8', 115200, timeout=1)
time.sleep(2)  # Wait for the port to initialize

# Send data
message = extracted_text
ser.write(message.encode())
print(f"Sent: {message.strip()}")   
ser.close()

Sent: BH01LV1947
